<a href="https://colab.research.google.com/github/k1ran242/kiran/blob/main/Fashion_Mnist_CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 102kB 6.7MB/s 
  Created wheel for kt-legacy: filename=kt_legacy-1.0.3-cp37-none-any.whl size=9569 sha256=8c67243c7bf06bd97867789881bbf2d51d22fb9189e43071f16517a6ce85a6f6
  Stored in directory: /root/.cache/pip/wheels/95/99/a8/6b116b0f69be60cc475d1cd36680f3a09f284d86655bb99d93
Successfully built kt-legacy


In [26]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [27]:
print(tf.__version__)

2.5.0


In [28]:
data=keras.datasets.fashion_mnist

In [29]:
(train_images,train_labels),(test_images,test_labels)=data.load_data()

In [30]:
train_images=train_images/255.0
test_images=test_images/255.0

In [31]:
train_images.shape

(60000, 28, 28)

In [32]:
train_images[0].shape

(28, 28)

In [33]:
len(train_images)

60000

In [34]:
# Reshaping the data
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [35]:
train_images.shape

(60000, 28, 28, 1)

In [44]:
def build_model(hp):
  model=keras.Sequential([
  keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
        kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter',min_value=32,max_value=128,step=16),
        kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_unit',min_value=32,max_value=128,step=16),
        activation='relu'
    ),
    keras.layers.Dense(10,activation='softmax')

  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',metrics=['accuracy'])
  return model

In [45]:
from kerastuner import RandomSearch

In [46]:
from kerastuner.engine.hyperparameters import HyperParameters

In [47]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,
                          directory='output',
                          project_name='Fashion Mnist')

In [48]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 21s]
val_accuracy: 0.8766666650772095

Best val_accuracy So Far: 0.9123333096504211
Total elapsed time: 00h 03m 35s
INFO:tensorflow:Oracle triggered exit


In [49]:
model=tuner_search.get_best_models(num_models=1)[0]

In [50]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        48432     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3539072   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 3,589,914
Trainable params: 3,589,914
Non-trainable params: 0
_________________________________________________________________


In [52]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1142 - accuracy: 0.9576 - val_loss: 0.2751 - val_accuracy: 0.9138
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0759 - accuracy: 0.9723 - val_loss: 0.3014 - val_accuracy: 0.9167
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0500 - accuracy: 0.9821 - val_loss: 0.3596 - val_accuracy: 0.9122
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0356 - accuracy: 0.9868 - val_loss: 0.4406 - val_accuracy: 0.9082
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0262 - accuracy: 0.9907 - val_loss: 0.4283 - val_accuracy: 0.9155
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0263 - accuracy: 0.9914 - val_loss: 0.4678 - val_accuracy: 0.9198
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0191 - accuracy: 0.9935 - val_loss: 0.5373 - val_accuracy